In [2]:
from xai.dqn import DQN
from xai.stream import Stream
from xai.bytes import GigaBytes
     

dqn = DQN(translate=True, rotate=True, device="cpu")     

In [3]:
dqn.train(
    num_episodes=10000,
    update_target_frequency=10,
    epsilon=.99,
    gamma=.98,
    learning_starts=0.1,
    batch_size=32,
    buffer_entry_size=3_000,
    replay_buffer_memory=GigaBytes(5.0)
)

RAM used: 0.12/5.00GB: : 301it [00:01, 292.92it/s]eward=160: : 1it [00:00,  2.40it/s]
Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
RAM used: 0.24/5.00GB: : 301it [00:00, 304.41it/s]
episode=1, replay_buffer.entry_size()=602, episode_reward=290: : 2it [03:46, 113.34s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/eivind/master-project/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3163/2588240655.py", line 1, in <module>
    dqn.train(
  File "/home/eivind/master-project/xai/dqn.py", line 218, in train
    batch = replay_buffer.randoms(with_replacement=False).take(batch_size).tuple()
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/eivind/master-project/xai/dqn.py", line 218, in <listcomp>
    batch = replay_buffer.randoms(with_replacement=False).take(batch_size).tuple()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/eivind/master-project/xai/dqn.py", line 98, in predict
  File "/home/eivind/master-project/xai/dqn.py", line 72, in state
    observation = observation.translated()
                  ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/eivind/mast

In [ ]:
dqn.play(show=True, show_decode=True)

In [ ]:
import torch

X = torch.zeros((1,3,210,160)).float()

policy = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3,3), stride=(1,1)), # (102, 76)
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(5,5), stride=(2,2)), # (107, 76)
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(7,7), stride=(3,3)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(7,7), stride=(4,4)),
    torch.nn.ReLU(),
    )

policy(X).shape

In [ ]:
64*7*5